# XGBoost Model for Spare Part Demand Forecasting

This notebook implements XGBoost for short-term demand forecasting (1-14 days).

## Objectives
1. Load and prepare data with feature engineering
2. Train XGBoost model
3. Hyperparameter tuning
4. Evaluate model performance
5. Feature importance analysis
6. Compare with Prophet

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded successfully!')
print(f'XGBoost version: {xgb.__version__}')

Libraries loaded successfully!
XGBoost version: 3.1.2


## 1. Load Data

In [2]:
# Load daily aggregated demand
df = pd.read_csv('../data/processed/daily_demand.csv', parse_dates=['date'])
print(f'Loaded {len(df)} rows')
df.head()

Loaded 730 rows


,date,demand_quantity,revenue
0,2022-01-01,5809,15363979.92
1,2022-01-02,5773,15295649.75
2,2022-01-03,14097,37141288.42
3,2022-01-04,9586,25126363.50
4,2022-01-05,9496,24857942.96


## 2. Feature Engineering

In [3]:
# Create date-based features
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_month'] = df['date'].dt.day
df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['year'] = df['date'].dt.year
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)

print('Date features created!')
df.head()

Date features created!


,date,demand_quantity,revenue,day_of_week,day_of_month,week_of_year,month,quarter,year,is_weekend,is_month_start,is_month_end
0,2022-01-01,5809,15363979.92,5,1,52,1,1,2022,1,1,0
1,2022-01-02,5773,15295649.75,6,2,52,1,1,2022,1,0,0
2,2022-01-03,14097,37141288.42,0,3,1,1,1,2022,0,0,0
3,2022-01-04,9586,25126363.50,1,4,1,1,1,2022,0,0,0
4,2022-01-05,9496,24857942.96,2,5,1,1,1,2022,0,0,0


In [4]:
# Create lag features
lags = [1, 7, 14, 30]
for lag in lags:
    df[f'lag_{lag}'] = df['demand_quantity'].shift(lag)

print(f'Lag features created: {lags}')
df.head(35)

Lag features created: [1, 7, 14, 30]


,date,demand_quantity,revenue,day_of_week,day_of_month,week_of_year,month,quarter,year,is_weekend,is_month_start,is_month_end,lag_1,lag_7,lag_14,lag_30
0,2022-01-01,5809,15363979.92,5,1,52,1,1,2022,1,1,0,NaN,NaN,NaN,NaN
1,2022-01-02,5773,15295649.75,6,2,52,1,1,2022,1,0,0,5809.0,NaN,NaN,NaN
2,2022-01-03,14097,37141288.42,0,3,1,1,1,2022,0,0,0,5773.0,NaN,NaN,NaN
3,2022-01-04,9586,25126363.50,1,4,1,1,1,2022,0,0,0,14097.0,NaN,NaN,NaN
4,2022-01-05,9496,24857942.96,2,5,1,1,1,2022,0,0,0,9586.0,NaN,NaN,NaN
5,2022-01-06,9514,24868081.48,3,6,1,1,1,2022,0,0,0,9496.0,NaN,NaN,NaN
6,2022-01-07,10475,27419333.52,4,7,1,1,1,2022,0,0,0,9514.0,NaN,NaN,NaN
7,2022-01-08,5716,15079513.98,5,8,1,1,1,2022,1,0,0,10475.0,5809.0,NaN,NaN
8,2022-01-09,5816,15443081.83,6,9,1,1,1,2022,1,0,0,5716.0,5773.0,NaN,NaN
9,2022-01-10,9585,25238009.28,0,10,2,1,1,2022,0,0,0,5816.0,14097.0,NaN,NaN


In [5]:
# Create rolling features
windows = [7, 14, 30]
for window in windows:
    df[f'rolling_mean_{window}'] = df['demand_quantity'].shift(1).rolling(window=window).mean()
    df[f'rolling_std_{window}'] = df['demand_quantity'].shift(1).rolling(window=window).std()

print(f'Rolling features created for windows: {windows}')
df.tail()

Rolling features created for windows: [7, 14, 30]


,date,demand_quantity,revenue,day_of_week,day_of_month,week_of_year,month,quarter,year,is_weekend,...,lag_1,lag_7,lag_14,lag_30,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30
725,2023-12-27,9230,24099746.28,2,27,52,12,4,2023,0,...,9303.0,9246.0,9218.0,9509.0,8380.142857,1873.757582,8350.642857,1797.133556,8441.633333,1737.531513
726,2023-12-28,9284,24183137.66,3,28,52,12,4,2023,0,...,9230.0,9320.0,9317.0,9530.0,8377.857143,1872.534684,8351.500000,1797.581869,8432.333333,1732.360485
727,2023-12-29,10302,27036073.93,4,29,52,12,4,2023,0,...,9284.0,10164.0,10103.0,9420.0,8372.714286,1869.562936,8349.142857,1796.239570,8424.133333,1727.561162
728,2023-12-30,5682,14838505.08,5,30,52,12,4,2023,1,...,10302.0,5692.0,5526.0,9602.0,8392.428571,1892.190605,8363.357143,1811.905143,8453.533333,1752.419623
729,2023-12-31,5720,15012004.56,6,31,52,12,4,2023,1,...,5682.0,5664.0,5738.0,10130.0,8391.000000,1894.571456,8374.500000,1793.499920,8322.866667,1809.062145


In [6]:
# Drop rows with NaN values (from lag and rolling features)
df_clean = df.dropna().copy()
print(f'Rows after dropping NaN: {len(df_clean)} (dropped {len(df) - len(df_clean)})')

Rows after dropping NaN: 700 (dropped 30)


In [7]:
# Check all features
print('Final features:')
print(df_clean.columns.tolist())

Final features:
['date', 'demand_quantity', 'revenue', 'day_of_week', 'day_of_month', 'week_of_year', 'month', 'quarter', 'year', 'is_weekend', 'is_month_start', 'is_month_end', 'lag_1', 'lag_7', 'lag_14', 'lag_30', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30']


## 3. Train-Test Split

In [8]:
# Define target and features
target = 'demand_quantity'
exclude_cols = ['date', 'demand_quantity', 'revenue']
feature_cols = [col for col in df_clean.columns if col not in exclude_cols]

print(f'Target: {target}')
print(f'Features ({len(feature_cols)}): {feature_cols}')

Target: demand_quantity
Features (19): ['day_of_week', 'day_of_month', 'week_of_year', 'month', 'quarter', 'year', 'is_weekend', 'is_month_start', 'is_month_end', 'lag_1', 'lag_7', 'lag_14', 'lag_30', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30']


In [9]:
# Time series split (no shuffling!)
test_days = 30
train_df = df_clean[:-test_days]
test_df = df_clean[-test_days:]

X_train = train_df[feature_cols]
y_train = train_df[target]
X_test = test_df[feature_cols]
y_test = test_df[target]

print(f'Training set: {X_train.shape}')
print(f'Test set: {X_test.shape}')

Training set: (670, 19)
Test set: (30, 19)


## 4. Train XGBoost Model

In [10]:
# Initialize XGBoost model
model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective='reg:squarederror'
)

print('XGBoost model initialized!')

XGBoost model initialized!


In [11]:
# Train the model
print('Training XGBoost model...')
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)
print('Model trained successfully!')

Training XGBoost model...
Model trained successfully!


## 5. Model Evaluation

In [12]:
# Generate predictions
y_pred = model.predict(X_test)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

print('='*50)
print('XGBOOST MODEL EVALUATION METRICS')
print('='*50)
print(f'MAE  (Mean Absolute Error):     {mae:.2f}')
print(f'RMSE (Root Mean Squared Error): {rmse:.2f}')
print(f'MAPE (Mean Absolute % Error):   {mape:.2f}%')
print(f'R2   (R-Squared):               {r2:.4f}')
print('='*50)

XGBOOST MODEL EVALUATION METRICS
MAE  (Mean Absolute Error):     507.09
RMSE (Root Mean Squared Error): 716.10
MAPE (Mean Absolute % Error):   5.96%
R2   (R-Squared):               0.8426


In [13]:
# Visualize actual vs predicted
results_df = pd.DataFrame({
    'Date': test_df['date'].values,
    'Actual': y_test.values,
    'Predicted': y_pred
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df['Date'], y=results_df['Actual'],
                         mode='lines+markers', name='Actual', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=results_df['Date'], y=results_df['Predicted'],
                         mode='lines+markers', name='Predicted', line=dict(color='orange')))

fig.update_layout(title='XGBoost: Actual vs Predicted (Test Period)',
                  xaxis_title='Date', yaxis_title='Demand')
fig.show()

## 6. Feature Importance

In [14]:
# Get feature importance
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print('Top 10 Features:')
importance_df.head(10)

Top 10 Features:


,Feature,Importance
6,is_weekend,0.596265
0,day_of_week,0.162672
15,rolling_mean_14,0.052120
13,rolling_mean_7,0.042349
11,lag_14,0.032026
10,lag_7,0.031364
17,rolling_mean_30,0.016380
2,week_of_year,0.010187
9,lag_1,0.008961
14,rolling_std_7,0.008182


In [15]:
# Visualize feature importance
fig = px.bar(importance_df.head(15), x='Importance', y='Feature', orientation='h',
             title='XGBoost Feature Importance (Top 15)',
             color='Importance', color_continuous_scale='Oranges')
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

## 7. Hyperparameter Tuning

In [16]:
# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.15]
}

# Time series cross-validation
tscv = TimeSeriesSplit(n_splits=3)

# Grid search
print('Running hyperparameter tuning (this may take a few minutes)...')
grid_search = GridSearchCV(
    xgb.XGBRegressor(random_state=42, objective='reg:squarederror'),
    param_grid,
    cv=tscv,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)
print('Tuning complete!')

Running hyperparameter tuning (this may take a few minutes)...
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Tuning complete!


In [17]:
# Best parameters
print('Best Parameters:')
print(grid_search.best_params_)
print(f'\nBest CV Score (MAE): {-grid_search.best_score_:.2f}')

Best Parameters:
{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 50}

Best CV Score (MAE): 1102.04


In [18]:
# Train final model with best parameters
best_model = grid_search.best_estimator_

# Evaluate on test set
y_pred_best = best_model.predict(X_test)

mae_best = mean_absolute_error(y_test, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
mape_best = np.mean(np.abs((y_test - y_pred_best) / y_test)) * 100

print('='*50)
print('TUNED XGBOOST MODEL METRICS')
print('='*50)
print(f'MAE:  {mae_best:.2f} (vs {mae:.2f} before tuning)')
print(f'RMSE: {rmse_best:.2f} (vs {rmse:.2f} before tuning)')
print(f'MAPE: {mape_best:.2f}% (vs {mape:.2f}% before tuning)')
print('='*50)

TUNED XGBOOST MODEL METRICS
MAE:  501.58 (vs 507.09 before tuning)
RMSE: 543.66 (vs 716.10 before tuning)
MAPE: 6.68% (vs 5.96% before tuning)


## 8. Cross-Validation

In [19]:
# Perform cross-validation on full dataset
tscv = TimeSeriesSplit(n_splits=5)

X_full = df_clean[feature_cols]
y_full = df_clean[target]

cv_scores = cross_val_score(best_model, X_full, y_full, cv=tscv, scoring='neg_mean_absolute_error')

print('Cross-Validation Results (5-fold TimeSeriesSplit):')
print(f'MAE Scores: {-cv_scores}')
print(f'Mean MAE: {-cv_scores.mean():.2f} (+/- {cv_scores.std():.2f})')

Cross-Validation Results (5-fold TimeSeriesSplit):
MAE Scores: [1221.73364258 1331.57055664  941.25518799  585.42071533  634.52740479]
Mean MAE: 942.90 (+/- 300.57)


## 9. Save Model

In [20]:
# Save the trained model
import os
os.makedirs('../models', exist_ok=True)

model_data = {
    'model': best_model,
    'feature_names': feature_cols,
    'params': grid_search.best_params_
}

model_path = '../models/xgboost_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model_data, f)

print(f'Model saved to: {model_path}')

Model saved to: ../models/xgboost_model.pkl


In [21]:
# Save metrics for comparison
metrics = {
    'model': 'XGBoost',
    'mae': mae_best,
    'rmse': rmse_best,
    'mape': mape_best,
    'cv_mae_mean': -cv_scores.mean()
}

metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('../models/xgboost_metrics.csv', index=False)
print('Metrics saved!')
metrics_df

Metrics saved!


,model,mae,rmse,mape,cv_mae_mean
0,XGBoost,501.577728,543.657233,6.679078,942.901501


## 10. Compare with Prophet

In [22]:
# Load Prophet metrics if available
try:
    prophet_metrics = pd.read_csv('../models/prophet_metrics.csv')
    xgb_metrics = pd.DataFrame([metrics])
    
    comparison = pd.concat([prophet_metrics, xgb_metrics], ignore_index=True)
    print('Model Comparison:')
    print(comparison.to_string(index=False))
    
    # Determine winner
    if comparison.loc[comparison['model'] == 'XGBoost', 'mae'].values[0] < comparison.loc[comparison['model'] == 'Prophet', 'mae'].values[0]:
        print('\n[WINNER] XGBoost has lower MAE!')
    else:
        print('\n[WINNER] Prophet has lower MAE!')
        
except FileNotFoundError:
    print('Prophet metrics not found. Run Prophet notebook first for comparison.')

Model Comparison:
  model        mae       rmse     mape  cv_mape_mean  cv_mae_mean
Prophet 464.641237 491.740794 6.203500      0.070244          NaN
XGBoost 501.577728 543.657233 6.679078           NaN   942.901501

[WINNER] Prophet has lower MAE!


In [23]:
# Visual comparison
try:
    fig = go.Figure(data=[
        go.Bar(name='Prophet', x=['MAE', 'RMSE', 'MAPE'], 
               y=[prophet_metrics['mae'].values[0], prophet_metrics['rmse'].values[0], prophet_metrics['mape'].values[0]],
               marker_color='#3B82F6'),
        go.Bar(name='XGBoost', x=['MAE', 'RMSE', 'MAPE'], 
               y=[mae_best, rmse_best, mape_best],
               marker_color='#F97316')
    ])
    fig.update_layout(title='Model Comparison: Prophet vs XGBoost', barmode='group')
    fig.show()
except:
    print('Could not create comparison chart.')

## Summary

| Metric | Value |
|--------|-------|
| Model | XGBoost |
| Training Period | ~670 days |
| Test Period | 30 days |
| Features | 20+ (lag, rolling, date features) |
| MAE | See above |
| RMSE | See above |
| MAPE | See above |

**Notes:**
- XGBoost uses feature engineering (lags, rolling stats)
- Hyperparameter tuning with GridSearchCV
- Best for short-term forecasts (1-14 days)
- Model saved for deployment

In [24]:
print('XGBoost Model Training Complete!')

XGBoost Model Training Complete!
